In [1]:
############ IMPORTS ####################
import sys
sys.path.append("../")
import os
import numpy as np
from scipy import io
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import torch
from operator import truediv
import record
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.losses import categorical_crossentropy
from keras.layers import Dense, Flatten, LSTM
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical as keras_to_categorical
import tensorflow as tf


############# CONFIGS ##########################

os.environ["CUDA_VISIBLE_DEVICES"]="1"
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

datasetNames = ["Trento"]
patchsize1 = 11
patchsize2 = 11
batchsize = 128
EPOCH = 4
LR = 0.001


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100

def get_model_compiled(feat_size,seq, Classes):
    func = LSTM
    clf = Sequential()
    clf.add(func(64, return_sequences=True, input_shape=(feat_size, seq)))
    clf.add(func(64, return_sequences=True))
    clf.add(Flatten())
    clf.add(Dense(Classes, activation='softmax'))
    clf.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-07,amsgrad=False), metrics=['accuracy'])
    return clf
        
for datasetName in datasetNames:
    print("----------------------------------Training for ",datasetName,"---------------------------------------------")
    
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    
    # Train data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands
    
    label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']
    
    # Test data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)
    
    label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']

    
    TrainPatch1 = torch.from_numpy(TrainPatch)
#         TrainPatch1 = TrainPatch1.permute(0,3,1,2)
    TrainLabel1 = torch.from_numpy(TrLabel)-1
    TrainLabel1 = TrainLabel1.long()
    

    TestPatch1 = torch.from_numpy(TestPatch)
#         TestPatch1 = TestPatch1.permute(0,3,1,2)
    TestLabel1 = torch.from_numpy(TsLabel)-1
    TestLabel1 = TestLabel1.long()

    Classes = len(np.unique(TrainLabel1))
    TrainPatch1 = TrainPatch1.reshape(TrainPatch1.shape[0],TrainPatch1.shape[1]*TrainPatch1.shape[2],TrainPatch1.shape[3])
    TestPatch1 = TestPatch1.reshape(TestPatch1.shape[0],TestPatch1.shape[1]*TestPatch1.shape[2],TestPatch1.shape[3])
    print("Train data shape = ", TrainPatch1.shape)
    print("Train label shape = ", TrainLabel1.shape)
    print("Test data shape = ", TestPatch1.shape)
    print("Test label shape = ", TestLabel1.shape)
        
    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((3, Classes))

    for iter in range(3):
        n_bands, sequences = TrainPatch1.shape[1:]
        clf = get_model_compiled(n_bands, sequences, Classes)
        valdata = (TestPatch1.cpu().detach().numpy(), keras_to_categorical(TestLabel1.reshape(-1).cpu().detach().numpy(),Classes))
        clf.fit(TrainPatch1.cpu().detach().numpy(), keras_to_categorical(TrainLabel1.reshape(-1).cpu().detach().numpy(),Classes),
                            batch_size=batchsize,
                            epochs=EPOCH,
                            verbose=True,
                            validation_data=valdata,
                            callbacks = [ModelCheckpoint(datasetName+"/best_model_HSIOnly.h5", monitor='val_accuracy', verbose=0, save_best_only=True)])



        clf = load_model(datasetName+"/best_model_HSIOnly.h5")
        pred_y = np.argmax(clf.predict(TestPatch1.cpu().detach().numpy()), axis=1)

        y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
        oa = accuracy_score(y_test, pred_y)*100
        confusion = confusion_matrix(y_test, pred_y)
        each_acc, aa = AA_andEachClassAccuracy(confusion)
        kappa = cohen_kappa_score(y_test, pred_y)*100
        KAPPA.append(kappa)
        OA.append(oa)
        AA.append(aa)
        ELEMENT_ACC[iter, :] = each_acc
    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/LSTM_Report_' + datasetName +'.txt')

    

2021-10-30 09:00:09.619182: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.2
2021-10-30 09:00:11.529407: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-10-30 09:00:11.559982: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1564] Found device 0 with properties: 
pciBusID: 0004:05:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.50GiB deviceMemoryBandwidth: 836.37GiB/s
2021-10-30 09:00:11.560016: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.2
2021-10-30 09:00:11.560082: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-10-30 09:00:11.560125: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.

----------------------------------Training for  Trento ---------------------------------------------
Train data shape =  torch.Size([819, 11, 11, 63])
Train label shape =  torch.Size([1, 819])
Test data shape =  torch.Size([29395, 11, 11, 63])
Test label shape =  torch.Size([1, 29395])


2021-10-30 09:00:12.430489: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3783000000 Hz
2021-10-30 09:00:12.505638: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x114e4bb10 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-30 09:00:12.505664: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-30 09:00:12.576983: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x114eb6ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-10-30 09:00:12.577019: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2021-10-30 09:00:12.578331: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1564] Found device 0 with properties: 
pciBusID: 0004:05:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80

Epoch 1/4
7/7 [==============================] - 2s 275ms/step - loss: 1.6177 - accuracy: 0.3138 - val_loss: 1.2757 - val_accuracy: 0.4520
Epoch 2/4
7/7 [==============================] - 1s 144ms/step - loss: 1.1562 - accuracy: 0.4884 - val_loss: 0.9708 - val_accuracy: 0.4999
Epoch 3/4
7/7 [==============================] - 1s 160ms/step - loss: 0.8573 - accuracy: 0.6838 - val_loss: 0.7413 - val_accuracy: 0.7191
Epoch 4/4
7/7 [==============================] - 1s 182ms/step - loss: 0.5942 - accuracy: 0.8413 - val_loss: 0.5576 - val_accuracy: 0.8067
Epoch 1/4
7/7 [==============================] - 2s 263ms/step - loss: 1.6069 - accuracy: 0.2735 - val_loss: 1.2134 - val_accuracy: 0.7180
Epoch 2/4
7/7 [==============================] - 1s 179ms/step - loss: 1.1339 - accuracy: 0.5385 - val_loss: 0.9821 - val_accuracy: 0.6811
Epoch 3/4
7/7 [==============================] - 1s 186ms/step - loss: 0.8439 - accuracy: 0.7534 - val_loss: 0.6634 - val_accuracy: 0.8274
Epoch 4/4
7/7 [============